In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from parse import parse

# Initializing driver
driver = webdriver.Firefox()
driver.get("https://coursefeedback.uchicago.edu/?Department=CMSC&AcademicYear=2022&AcademicTerm=Spring")
time.sleep(35)

# Enter your credientials here

# Get all courses in the search
driver.get("https://coursefeedback.uchicago.edu/?Department=CMSC&AcademicYear=2022&AcademicTerm=Spring")
courses = driver.find_elements(By.CSS_SELECTOR, "td[class='course']>a")

links = []
for course in courses:
    links.append(course.get_attribute("href"))


In [21]:
# Get one evaluation 
evaluation = {}
driver.get(links[0])
time.sleep(3)

# Get eval metadata
metadata = {}
report_metadata = driver.find_element(By.CLASS_NAME, "metadata")

report_title = report_metadata.find_element(By.TAG_NAME, "h2").text
course_num, course_name, instructors = parse("{} - {} - Instructor(s): {}", report_title)
metadata["Catalog Number"] = course_num.split(",")
metadata["Course Name"] = course_name
metadata["Instructors"] = instructors.split(",")

offer_time, num_enrolled, num_respond = report_metadata.find_elements(By.TAG_NAME, "dd")[:3]
term, year = offer_time.text.split(" - ")[1].split(" ")
metadata["Academic Year"] = term.strip()
metadata["Academic Term"] = year.strip()
metadata["Number Enrolled"] = num_enrolled.text.strip()
metadata["Number Responses"] = num_respond.text.strip()


evaluation["Evaluation Metadata"] = metadata

# get the questions and answers
text_data = {}
reports = driver.find_elements(By.CLASS_NAME, "report-block")
for report in reports:
    question = report.find_element(By.TAG_NAME, "span").text


    comments = []
    report_comments = report.find_elements(By.TAG_NAME, "tr")
    if (len(report_comments)) == 0:
        continue
    for comment in report_comments[1:]:
        comments.append(comment.text)

    text_data[question] = comments

evaluation["Evaluation Text Data"] = text_data

evaluation

{'Evaluation Metadata': {'Catalog Number': ['DATA 11900 1',
   ' CMSC 11900 1',
   ' STAT 11900 1'],
  'Course Name': 'Introduction to Data Science II',
  'Instructors': ['William Trimble'],
  'Academic Year': 'Spring',
  'Academic Term': '2023',
  'Number Enrolled': '54',
  'Number Responses': '20'},
 'Evaluation Text Data': {'What are the most important things that you learned in this course? Please reflect on the knowledge and skills you gained.': ['A random agglomeration of loosely related data science topics.',
   'I learned how to teach myself various regression and regularization techniques.',
   'I learned about regressions and how to determine causality between variables.',
   'Learning to Visualize data correctly using graphs and such',
   'I learned different methods of analysis for effect in the context of varying experimental designs.',
   '–better data visualization\n–types of regression\n–neural networks\n–basic SQL',
   'Multiple linear regression, binary classification